In [1]:
import os
import pickle
import itertools
import pandas as pd
import numpy as np
#from modelgym import model
import functools
import modelgym
from modelgym.util import TASK_CLASSIFICATION
from modelgym.trainer import Trainer
from modelgym.tracker import ProgressTrackerFile, ProgressTrackerMongo
from sklearn.metrics import roc_auc_score
from hyperopt.mongoexp import MongoTrials
from modelgym.util import split_and_preprocess
from sklearn.model_selection import train_test_split
from collections import OrderedDict

In [2]:
########### NROWS, N_ESTIMATORS, N_PROBES, TEST_SIZE, N_CV_SPLITS, OPTIMIZER
config_tuple = {
    'test': (1000, 100,  2, 0.5, 2, 'random'),
    'pror': (None, 1000, 100, 0.5, 2, 'random'), # production with random hyperopt suggestor
    'prot': (None, 1000, 100, 0.5, 2, 'tpe'),    # production with tpe hyperopt suggestor
    'demi': (10000, 100, 5, 0.5, 2, 'random')
}
CONFIG = 'test' if 'EXP_CONFIG' not in os.environ else os.environ['EXP_CONFIG']
NROWS, N_ESTIMATORS, N_PROBES, TEST_SIZE, N_CV_SPLITS, OPTIMIZER = config_tuple[CONFIG]
CANDIDATES = OrderedDict([
    ('XGBoost', modelgym.XGBModel), 
    ('LightGBM', modelgym.LGBModel),
    ('RandomForestClassifier',modelgym.RFModel)
])
RESULTS_DIR = "results"
LOAD_CACHE = False
if 'MONGO_PORT_27017_TCP_ADDR' in os.environ:
    mongo_host = os.environ['MONGO_PORT_27017_TCP_ADDR'] if 'MONGO_PORT_27017_TCP_ADDR' in os.environ else 'cern-mc01h'
    mongo_port = int(os.environ['MONGO_PORT_27017_TCP_PORT']) if 'MONGO_PORT_27017_TCP_PORT' in os.environ else 27017
    mongo_db = os.environ['MONGO_DB'] if 'MONGO_DB' in os.environ else 'trials'
    tracker_factory = functools.partial(ProgressTrackerMongo, mongo_host, mongo_port, mongo_db, config_key=CONFIG)
    print ("Using Mongo as backend for tracking")
else:
    tracker_factory = functools.partial(ProgressTrackerFile, RESULTS_DIR, config_key=CONFIG)
    print ("Using File as backend for tracking")

print ("Running experiment cofiguration:", CONFIG)

Using File as backend for tracking
Running experiment cofiguration: test


## Download & read data file

In [3]:
%%bash 
if [ ! -d data ] ; then 
    mkdir data 
    cd data
    curl https://cernbox.cern.ch/index.php/s/N1dpSAPgl30szYM/download | gunzip -c > XY2d.pickle
    cd ..
fi
ls -l data

total 266224
-rw-r--r--  1 macbook  staff  136304022 Aug 11 14:40 XY2d.pickle


In [4]:
def read_data(fname, nrows=None, shuffle=True):
    with open(fname,'rb') as fh:
        X, y = pickle.load(fh,encoding='bytes')
    index = np.arange(X.shape[0])
    if nrows is None:
        nrows = X.shape[0]
    weights = np.ones(nrows) # uh, well...
    if shuffle:
        index_perm = np.random.permutation(index)
    else:
        index_perm = index
    return X[index_perm[:nrows]], y[index_perm[:nrows]], weights


X, y, weights = read_data("data/XY2d.pickle", nrows=NROWS)

In [5]:
X_train, X_test, y_train, y_test, w_train, w_test = train_test_split(X, y, weights, test_size=TEST_SIZE)

In [6]:
cv_pairs, (dtrain, dtest) = split_and_preprocess(X_train.copy(), y_train, 
                                                X_test.copy(), y_test, 
                                                cat_cols=[], n_splits=N_CV_SPLITS)

## Run them all

In [7]:
trackers = {}
def init_keys_dict():
    return dict([(k, None) for k in CANDIDATES.keys()])
default_cv_result = init_keys_dict()
tuned_cv_result = init_keys_dict()
default_test_result = init_keys_dict()
tuned_test_result = init_keys_dict()
trials = init_keys_dict()
trainer = Trainer(hyperopt_evals=N_PROBES, n_estimators=N_ESTIMATORS)

In [8]:
for model_id, model_class in CANDIDATES.items():
    model = model_class(TASK_CLASSIFICATION)
    print ("~"*20, model.get_name(), "~"*20)
    trackers[model_id] = tracker_factory(model_name=model.get_name())
    if LOAD_CACHE:
        default_cv_result[model_id], default_test_result[model_id], tuned_cv_result[model_id], tuned_test_result[model_id], trials[model_id] = \
            trackers[model_id].load_state(as_list=True)
    
    
    if default_cv_result[model_id] is None:
        default_cv_result[model_id] = trainer.crossval_fit_eval(model, cv_pairs)
        trackers[model_id].save_state(default_cv=default_cv_result[model_id])
    trainer.print_result(default_cv_result[model_id], 'Default {} result on CV'.format(model.get_name()))

    if default_test_result[model_id] is None:
        default_test_result[model_id] = trainer.fit_eval(model, dtrain, dtest,
                                                  default_cv_result[model_id]['params'],
                                                  default_cv_result[model_id]['best_n_estimators'],
                                                  custom_metric = {'roc_auc': roc_auc_score})
        trackers[model_id].save_state(default_test=default_test_result[model_id])

    trainer.print_result(default_test_result[model_id], 'Default {} result on TEST'.format(model.get_name()), extra_keys=['roc_auc'])

        
    if tuned_cv_result[model_id] is None:
        print('Hyperopt iterations:\n\n')
        tuned_cv_result[model_id] = trainer.crossval_optimize_params(model, cv_pairs,  algo_name=OPTIMIZER, 
                                                           trials=trials[model_id], tracker=trackers[model_id])
        trackers[model_id].save_state(tuned_cv=tuned_cv_result[model_id])
    trainer.print_result(tuned_cv_result[model_id], 'Tuned {} result on cv'.format(model.get_name()))

    if tuned_test_result[model_id] is None:
        tuned_test_result[model_id] = trainer.fit_eval(model, dtrain, dtest,
                                            tuned_cv_result[model_id]['params'],
                                            tuned_cv_result[model_id]['best_n_estimators'],
                                            custom_metric = {'roc_auc': roc_auc_score})
        trackers[model_id].save_state(tuned_test=tuned_test_result[model_id])
    trainer.print_result(tuned_test_result[model_id], 'Tuned {} result on test'.format(model.get_name()), extra_keys=['roc_auc'])

    trackers[model_id].save_state(default_cv=default_cv_result[model_id], default_test=default_test_result[model_id], 
                               tuned_cv=tuned_cv_result[model_id], tuned_test=tuned_test_result[model_id], trials=trials[model_id])
    

~~~~~~~~~~~~~~~~~~~~ XGBoost ~~~~~~~~~~~~~~~~~~~~
BST  <xgboost.core.Booster object at 0x10f2d2898>
RES [0.639015, 0.59697, 0.560292, 0.530095, 0.504147, 0.483146, 0.466197, 0.45078, 0.43739, 0.425455, 0.41608, 0.4089, 0.403945, 0.398489, 0.393422, 0.39033, 0.389786, 0.386504, 0.385315, 0.384363, 0.381574, 0.381022, 0.378459, 0.377447, 0.37765, 0.378634, 0.378764, 0.381958, 0.383001, 0.381755, 0.38291, 0.383754, 0.388388, 0.389157, 0.390195, 0.389849, 0.391495, 0.392524, 0.394254, 0.395603, 0.395377, 0.395017, 0.39507, 0.399614, 0.399553, 0.401836, 0.401376, 0.40281, 0.404191, 0.405153, 0.406114, 0.407608, 0.408868, 0.410935, 0.410542, 0.412306, 0.413515, 0.41356, 0.413748, 0.41598, 0.415033, 0.416508, 0.415067, 0.416609, 0.415681, 0.415448, 0.415255, 0.416849, 0.417567, 0.416777, 0.418229, 0.416653, 0.41775, 0.418928, 0.419566, 0.420083, 0.418683, 0.415769, 0.41768, 0.419547, 0.419845, 0.421107, 0.421911, 0.423609, 0.424732, 0.425194, 0.425876, 0.425704, 0.426431, 0.426193, 0.425691, 

roc_auc = 0.821401
saved state to results/tracker_test_XGBoost.pickle
~~~~~~~~~~~~~~~~~~~~ LightGBM ~~~~~~~~~~~~~~~~~~~~
saved state to results/tracker_test_LightGBM.pickle
Default LightGBM result on CV:

loss = 0.372338232802
best_n_estimators = 33
params = {'boosting_type': 'gbdt', 'colsample_bytree': 1, 'drop_rate': 0.1, 'is_unbalance': False, 'learning_rate': 0.1, 'max_bin': 255, 'min_data_in_leaf': 20, 'max_depth': -1, 'max_drop': 50, 'min_child_samples': 10, 'min_child_weight': 5, 'min_split_gain': 0, 'min_sum_hessian_in_leaf': 0.001, 'lambda_l1': 0, 'lambda_l2': 0, 'n_estimators': 10, 'nthread': 4, 'num_threads': 4, 'num_leaves': 31, 'reg_alpha': 0, 'reg_lambda': 0, 'scale_pos_weight': 1, 'seed': 0, 'sigmoid': 1.0, 'skip_drop': 0.5, 'subsample': 1, 'subsample_for_bin': 50000, 'subsample_freq': 1, 'uniform_drop': False, 'xgboost_dart_mode': False, 'objective': 'binary', 'metric': 'binary_logloss', 'bagging_freq': 1, 'verbose': -1}
saved state to results/tracker_test_LightGBM.pick

{'criterion': 'gini', 'max_depth': 19, 'max_features': 3, 'n_estimators': 15, 'normalize': 1, 'scale': 0}
{'max_depth': 1, 'max_features': 4, 'n_estimators': 10, 'criterion': 'gini', 'verbose': 0}
[0.84804490902051877, 0.85598141695702668, 0.8324622531939605, 0.83614014711575679, 0.84804490902051877, 0.86411149825783973, 0.86807975222609368, 0.83614014711575679, 0.85201316298877272, 0.80023228803716606, 0.66008517228029417, 0.86004645760743326, 0.84407665505226481, 0.85201316298877272, 0.80816879597367397, 0.73596593108788222, 0.86004645760743326, 0.84010840108401075, 0.81997677119628332, 0.84001161440185823, 0.84417344173441722, 0.85588463027487416, 0.84020518776616326, 0.83604336043360439, 0.86004645760743326, 0.8560782036391793, 0.85210994967092535, 0.83604336043360428, 0.80400696864111509, 0.8560782036391793, 0.84427022841656985, 0.84407665505226481, 0.86004645760743326, 0.86401471157568721, 0.86807975222609368, 0.85994967092528063, 0.85994967092528063, 0.8560782036391793, 0.840011

AttributeError: 'XYCDataset' object has no attribute 'get_label'

## Compare

In [ ]:
metric, mes_min = 'roc_auc', False
full_results = {}
for i in CANDIDATES.keys():
    if i in trackers:
        tracker = trackers[i]
    else:
        tracker = tracker_factory(model_name=i)
        tracker.load_state()
    full_results.update({i:{'tuned': tracker.state['tuned_test'], 'default': tracker.state['default_test']}})

In [ ]:
def plot_metric_results(full_results, index, metric, is_min_better=True):
    test_results_list = []
    for i in index:
        test_results_list.append([full_results[i]['default'][metric], full_results[i]['tuned'][metric]])
        
    test_results = np.array(test_results_list)
    if is_min_better:
        baseline = test_results.min()
    else:
        baseline = test_results.max()
    diff = 100 * test_results / baseline - 100
    test_results_formatted = [['{:.6f} ({:+.2f}%)'.format(test_results[i, j], diff[i, j]) for j in range(2)] for i in range(len(index))]

    print (pd.DataFrame(test_results_formatted, columns=['default', 'tuned'], index=index))
    
    full_names = [" ".join(i) for i in itertools.product(index, ['default', 'tuned'])]

    named_results = zip(full_names, test_results.flatten())

    sorted_results = sorted(named_results, key=lambda x: x[1], reverse=not is_min_better)
    xticks = ['%s\n%.5f' % (name, loss) for name, loss in sorted_results]

    pyplot.figure(figsize=(20, 7))
    pyplot.scatter(range(len(full_names)), list(zip(*sorted_results))[1], s=150)
    pyplot.xticks(range(len(full_names)), xticks, fontsize=15)
    pyplot.yticks(fontsize=12)
    pyplot.title('Comparison', fontsize=20)
    pyplot.ylabel(metric, fontsize=16)

In [ ]:
%pylab inline --no-import-all
metric, is_min_better = 'roc_auc', False
plot_metric_results(full_results, CANDIDATES.keys(), metric, is_min_better=is_min_better)